In [ ]:
from tools import get_soup

In [ ]:
url = 'https://www.imdb.com/chart/moviemeter/'

In [ ]:
# tools.py 未加headers不行，加了就OK了；

soup=get_soup(url)    #用網頁上文字，找找看有沒有在soup裡，確認OK
soup

### 目標
- 首頁：本週排名、上週排名、片名、連結
- 子頁：上映年、片長、類型、IMDb Rating、預告片連結、圖片連結

In [ ]:
# 先試掠才來再用findall
# 結果發現是字典格式（其實是json）
# 用.text共前後的script標籤提掉
mvs = soup.find('script', id='__NEXT_DATA__').text
mvs

In [ ]:
type(mvs)

In [ ]:
# 轉做json，用dumps（注意，有s）
# 愛編碼

import json

# 共str轉做dict
dmvs=json.loads(mvs)


In [ ]:
type(dmvs)

In [ ]:
# 用網頁kw像片名定位
# 發見片的連結無佇dict內底，soup內底有
# 問題是soup的href彼區無囥英語片名
# 看片名的構成，發現是 imdb.com/title/{id}

dmvs

In [ ]:
# 試看結構

dmvs['props']['pageProps']['initialRefinerQueryInfo']['queryContext']

In [ ]:
dmvs['props']['pageProps']

In [ ]:
# 揣著有關係彼層
data=dmvs['props']['pageProps']['pageData']['chartTitles']['edges']
data

In [ ]:
# 發現是list（佇dict內底的一个list元素）

type(data)

In [ ]:
# list內底每一支片是一个字典
data[0]

In [ ]:
# 攏總幾支片

len(data)

In [ ]:
#本週排名
cRank=data[0]['currentRank']
cRank
#上週排名
if data[0]['meterRanking']['rankChange']['changeDirection']=='UP':
    pRank=cRank+data[0]['meterRanking']['rankChange']['changeDirection']['difference']
elif data[0]['meterRanking']['rankChange']['changeDirection']=='DOWN':
    pRank=cRank-data[0]['meterRanking']['rankChange']['changeDirection']['difference']
else:
    pRank=cRank
pRank
#片名

#連結

In [ ]:
# 其他東西都在node之下

data[0]['node']

In [ ]:
#本週排名
cRank=data[0]['currentRank']
cRank
#上週排名
if data[0]['node']['meterRanking']['rankChange']['changeDirection']=='UP':
    pRank=cRank+data[0]['node']['meterRanking']['rankChange']['changeDirection']['difference']
elif data[0]['node']['meterRanking']['rankChange']['changeDirection']=='DOWN':
    pRank=cRank-data[0]['node']['meterRanking']['rankChange']['changeDirection']['difference']
else:
    pRank=cRank
pRank
#片名

#連結

In [ ]:
# 再觀察

data[0]['node']['meterRanking']

In [ ]:
#上週排名
if data[0]['node']['meterRanking']['rankChange']['changeDirection']=='UP':
    pRank=cRank+data[0]['node']['meterRanking']['rankChange']['difference']
elif data[0]['node']['meterRanking']['rankChange']['changeDirection']=='DOWN':
    pRank=cRank-data[0]['node']['meterRanking']['rankChange']['difference']
else:
    pRank=cRank
pRank


In [ ]:
#片名
title = data[0]['node']['originalTitleText']['text']
title


In [ ]:
#連結
num= data[0]['node']['id']
num

In [ ]:
#連結
link = 'https://imdb.com/title/'+num
print(link)
# 試連成功

In [ ]:
# 子頁：上映年、片長、類型、IMDb Rating、預告片連結、圖片連結
# 上映年、IMDb Rating、類型、圖片連結 頂懸的資料結構就有，先揣
#上映年
year = data[0]['node']['releaseYear']['year']
year


In [ ]:
#IMDb Rating
rating=data[0]['node']['ratingsSummary']['aggregateRating']
rating


In [ ]:
#類型；本來取[2]，後來發現毋是逐齣攏有[2]，改取[0]
genre=data[0]['node']['titleGenres']['genres'][0]['genre']['text']
genre


In [ ]:
#圖片連結
pic=genre=data[0]['node']['primaryImage']['url']
print(pic)

In [ ]:
# 入去子頁揣賰的：片長、預告片連結

subsoup=get_soup(link)
subsoup

In [ ]:
# 片長

ul=subsoup.find('ul',class_='ipc-inline-list ipc-inline-list--show-dividers sc-afe43def-4 kdXikI baseAlt')
length=ul.find_all('li',class_='ipc-inline-list__item')[2].text.strip()
length

In [ ]:
# 預告片連結
prem=subsoup.find('a',class_='ipc-lockup-overlay ipc-focusable').get('href')
prem

In [ ]:
# 預告片連結
print('https://imdb.com'+prem)

In [ ]:
# 準備將資料組合做一个list結構


#本週排名
cRank=data[0]['currentRank']
cRank

#上週排名
if data[0]['node']['meterRanking']['rankChange']['changeDirection']=='UP':
    pRank=cRank+data[0]['node']['meterRanking']['rankChange']['difference']
elif data[0]['node']['meterRanking']['rankChange']['changeDirection']=='DOWN':
    pRank=cRank-data[0]['node']['meterRanking']['rankChange']['difference']
else:
    pRank=cRank
pRank

#片名
title = data[0]['node']['originalTitleText']['text']
title

#連結
num= data[0]['node']['id']
num
link = 'https://imdb.com/title/'+num
print(link)

#上映年
year = data[0]['node']['releaseYear']['year']
year

#IMDb Rating
rating=data[0]['node']['ratingsSummary']['aggregateRating']
rating

#類型
genre=data[0]['node']['titleGenres']['genres'][0]['genre']['text']
genre

#圖片連結
pic=genre=data[0]['node']['primaryImage']['url']
print(pic)

# 片長
ul=subsoup.find('ul',class_='ipc-inline-list ipc-inline-list--show-dividers sc-afe43def-4 kdXikI baseAlt')
length=ul.find_all('li',class_='ipc-inline-list__item')[2].text.strip()
length

# 預告片連結
prem=subsoup.find('a',class_='ipc-lockup-overlay ipc-focusable').get('href')
premlink = 'https://imdb.com'+prem

In [ ]:
# 發現若有問題，攏是佇「片長」了後有問題，檢查結果，是「片長」彼位有時無「分級資訊」，所以會揣無[2]，愛用條件式判斷
# 先試片長改寫佇有問題的第23齣會用得袂

subsoup=get_soup('https://www.imdb.com/title/tt15654328/?ref_=chtmvm_t_23')
ul=subsoup.find('ul',class_='ipc-inline-list ipc-inline-list--show-dividers sc-afe43def-4 kdXikI baseAlt')
leng=ul.find_all('li',class_='ipc-inline-list__item')
if len(leng)==3:
    length=leng[2].text.strip()
else:
    length=leng[1].text.strip()
print(length)

In [ ]:
pop_movies = []

for i in range(0,100):   #攏總一百齣，對0開始算，算頭無算尾

# 把原來的[0]都換成[i]；中間攏加訊息，才袂攏毋知進度
# 加try, except，才袂斷去
    movie=[]

    try:
        print(f'這馬開始掠第{i+1}齣...')

        # 本周排名
        print(f'這馬咧掠第{i+1}齣目前排名！')
        cRank=data[i]['currentRank']
        movie.append(cRank)

        # 上周排名
        print(f'這馬咧掠第{i+1}齣進前的排名！')
        if data[i]['node']['meterRanking']['rankChange']['changeDirection']=='UP':
            pRank=cRank+data[0]['node']['meterRanking']['rankChange']['difference']
        elif data[i]['node']['meterRanking']['rankChange']['changeDirection']=='DOWN':
            pRank=cRank-data[i]['node']['meterRanking']['rankChange']['difference']
        else:
            pRank=cRank
        movie.append(pRank)

        #片名
        print(f'這馬咧掠第{i+1}齣的片名！')
        title = data[i]['node']['originalTitleText']['text']
        movie.append(title)

        #連結
        print(f'這馬咧掠第{i+1}齣的連結！')
        num= data[i]['node']['id']
        link = 'https://imdb.com/title/'+num
        movie.append(link)
        # 入去囝頁的兩項用link入去，愛佇遮處理
        subsoup=get_soup(link)
        # 片長
        # 發現若有問題，攏是佇「片長」了後有問題，檢查結果，是「片長」彼位有時無「分級資訊」，所以會揣無[2]，愛用條件式判斷
        print(f'這馬咧掠第{i+1}齣的片長！')
        ul=subsoup.find('ul',class_='ipc-inline-list ipc-inline-list--show-dividers sc-afe43def-4 kdXikI baseAlt')
        leng=ul.find_all('li',class_='ipc-inline-list__item')
        if len(leng)==3:
            length=leng[2].text.strip()
        else:
            length=leng[1].text.strip()
        movie.append(length)
        # 預告片連結
        print(f'這馬咧掠第{i+1}齣的預告片連結！')
        prem=subsoup.find('a',class_='ipc-lockup-overlay ipc-focusable').get('href')
        premlink = 'https://imdb.com'+prem
        movie.append(premlink)

        #上映年
        print(f'這馬咧掠第{i+1}齣的發行年度！')
        year = data[i]['node']['releaseYear']['year']
        movie.append(year)

        #IMDb Rating
        print(f'這馬咧掠第{i+1}齣的評分！')
        rating=data[i]['node']['ratingsSummary']['aggregateRating']
        movie.append(rating)

        #類型
        print(f'這馬咧掠第{i+1}齣的類型！')
        genre=data[i]['node']['titleGenres']['genres'][0]['genre']['text']
        movie.append(genre)

        #圖片連結
        print(f'這馬咧掠第{i+1}齣的圖片連結！')
        pic=genre=data[i]['node']['primaryImage']['url']
        movie.append(pic)

    except Exception as e:
        print(f'第{i+1}齣有問題！',e)
        pass

    pop_movies.append(movie)

In [ ]:
import pandas as pd

df = pd.DataFrame(pop_movies, columns=['current ranking','previous ranking','title','link','length','trailer','year','IMDb rating','genre','poster'])
df

In [ ]:
# 用今仔日的日期來寄檔名

from datetime import datetime

date=datetime.now().strftime('%Y-%m-%d')  #m愛細本字，因為大本字是指分鐘
date

df.to_csv(f'IMDb電影排行榜_{date}.csv',encoding='utf-8-sig')

In [ ]:
# 作業會當是：按怎連頭前掠原始碼敆做一个完整的程式
# 作業會當是：提供提示，去掠圖片！

### 儲存圖片
- 取得圖片二維位的數據
- 創建目錄
- 寫入檔案

In [ ]:
# 用自由時報頭頁做例

soup=get_soup('https://www.ltn.com.tw/')
soup


In [ ]:
#圖片標籤
pics=soup.find_all('img')
pics

In [ ]:
# 共圖片連結提出來
links=[]
for pic in pics:
    pic_link=pic.get('src')
    links.append(pic_link)

links

In [ ]:
len(links)

In [ ]:
# 用第一張圖做例

import requests     ##只要是超連結，都用requests先做

resp=requests.get(links[0])
resp

In [ ]:
"ltn.com" in 'assets/images/x.png'

In [ ]:
# 網站內底的連結攏無完整，愛家己加
# 有的連結完整，有的無，愛檢查

for link in links:
    if "ltn.com" not in link:
        link="https://www.ltn.com.tw/"+link
        print(link)

print(links)

# 按呢做無效！袂改，因為無入去替換，只是共印出來

In [ ]:
# 若欲會成，愛用一个新的list

new_links=[]

for link in links:
    if 'ltn.com' not in link:
        new_links.append('https://www.ltn.com.tw/'+link)
    else:
        new_links.append(link)

new_links

In [ ]:
# 以上單純練習，實際上觀察，原底就有ltn.com的才「有可能」是新聞圖片
# 干焦掠新聞圖片

news_pics=[]

for link in links:
    if 'ltn.com' in link:
        news_pics.append(link)

news_pics

In [ ]:
resp=requests.get(news_pics[0])
resp

In [ ]:
resp.content   #圖片是二進位制的，要用content

In [ ]:
#寫入模式wb，因不是文字，是二進位制檔
with open('1.jpg', 'wb') as f:
    f.write(resp.content)

### 一樣函式化，放入tools，就方便抓圖片

In [ ]:
def save_pic(url, filename='temp.jpg'):   #第二個參數可無
    try:
        resp=requests.get(url)
        if resp.status_code==200:
            with open(filename, 'wb') as f:
                f.write(resp.content)
                print(f'{filename}儲存完畢')
    except Exception as e:
        print('儲存失敗', resp.status_code)

In [ ]:
save_pic?
'''跑完後，發現Docstring裡沒有東西
在第一行下以長註解放基本資料就會有了'''

In [ ]:
save_pic(news_pics[8])    ##測試

### 儲存所有圖片
- 要讓filename不同，不然會一直覆蓋掉

In [ ]:
for i,pic in enumerate(news_pics):
    save_pic(pic, f'{i}.jpg')